In [1]:
import re
import csv
import os

In [2]:
def extract_data(path):
    lines_list = []
    lines = ''
    with open(path, 'r') as f:
        for line in f.readlines():
            if '>' not in line:
                line = line.replace('\n','').replace('\t','').replace('\r','').strip()
                lines = lines + line
            else:
                lines_list.append(lines)
                lines = ''
    lines_list.remove('')
    lines_list.append(lines)
    return lines_list

In [3]:
# 标准氨基酸 ACDEFGHIKLMNPQRSTVWY
# 当特征为连续k个时，创建k的n次方个特征返回
def create_features(new_features,k):
    row_features = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','U','X']
    length = len(new_features)
    features = []
    for i in range(length):
        for j in row_features:
            features.append(new_features[i] + j)
    if k > 1:
        k -= 1
        features = create_features(features,k)
    return features

In [4]:
# 特征工程，统计每条数据的每个特征出现的次数
def get_k_meter(data_list, label, k):
    new_data  = []
    features = create_features([''],k)
    for line in data_list:
        #print(line)
        data_line = {'label':label}
        for feature in features:
            data_line.setdefault(feature,0)
        length = len(line)-k +1
        for i in range(length):
            now_feature = line[i:i+k]
            data_line[now_feature] += 1
        new_data.append(data_line)
    features.insert(0,'label')
    print('特征生成完成')
    return new_data, features

In [5]:
# 将整合数据按行写入csv文件
def data_to_csv(positive_data,train_data_path_part1,train_data_path_part2, k ,features):
    train_data_path = train_data_path_part1 + str(k) +train_data_path_part2
    csv_file = open(train_data_path, 'a',newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(features)
    for line in positive_data:
        real_line = [line[i] for i in features]
        csv_writer.writerow(real_line)
    csv_file.close()
    print('文件整理输出完毕！！！')

In [6]:
# path
path_data_dir = os.path.dirname(os.getcwd())

path_negative_data = path_data_dir + r'\row_data\non-thermophilic.txt'
path_positive_data = path_data_dir +  r'\row_data\thermophilic.txt'

train_data_path_part1 = path_data_dir + r'\new_data\kmer\train_'
train_data_path_part2 = r'_meter.csv'

In [7]:
# 数据抽取并整合

for k in range(5,6):
    positive_data = extract_data(path_positive_data)
    negative_data = extract_data(path_negative_data)


    new_positive_data , features= get_k_meter(positive_data,'1',k)
    new_negative_data , features= get_k_meter(negative_data,'0',k)

    new_positive_data.extend(new_negative_data)

    print('{} mater数据整理完成，共 {} 条数据'.format(k,len(new_positive_data)))
    data_to_csv(new_positive_data,train_data_path_part1,train_data_path_part2, k ,features)


MemoryError: 